In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
%aimport AD_comparison_tools
%aimport AD_predictor_tools
%aimport ADpred_LambertTFs_helper
import protfasta
import matplotlib.pyplot as plt
import seaborn as sns
#import metapredict as meta
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, auc

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.lines import Line2D

sns.set_theme(rc={'figure.figsize':(6,4), 'figure.dpi' : 300})
sns.set_style("white")

from parrot import py_predictor

Error importing GPy.
 If trying to run parrot-optimize, make sure to use `pip install idptools-parrot[optimize]`


In [36]:
adhunter = pd.read_csv("../output/merged_ADHunter_Lambert_TF_predictions.csv", index_col = 0)
tada = pd.read_csv("../data/merged_TADA_Lambert_TF_predictions_0.4.csv", index_col = 0)
adpred = pd.read_csv("../output/predictions/adpred_LambertTF_predictions.csv", index_col = 0)
mechanistic = pd.read_csv("../output/predictions/LambertTFs_s_001_lcc_VP16_lch_006_ucc_-08_uch_CITED2_lcs1_inf_lcs2_000_lcs1_inf_ucs2_inf_comp_WFL_tl_039_ws_001_ps1_Charge_ps2_AllHydros", index_col = 0)
paddle = pd.read_csv("../data/Sanborn_HumanTF_Predictions_BothStrengths_Formatted",index_col=0)

adhunter = adhunter[["uniprotID", "Start", "End"]]
tada = tada[["uniprotID", "Start", "End"]]
adpred = adpred[["uniprotID", "Start", "End"]]
mechanistic["uniprotID"] = mechanistic["GeneName"].str.split("|").str[1]
mechanistic = mechanistic[["uniprotID", "Start", "End"]]
paddle = paddle[["uniprotID", "Start", "End"]]

display(adhunter)
display(tada)
display(adpred)
display(mechanistic)
display(paddle)

,uniprotID,Start,End
0,Q96QS3,393,436
1,P31270,0,47
2,P31270,110,159
3,P50221,56,99
4,P57073,245,324
...,...,...,...
1664,P17098,14,72
1665,Q9UJW7,11,73
1666,Q9UJW7,111,176
1667,Q9UJW7,187,256


,uniprotID,Start,End
0,P31270,1,41
1,P31270,108,170
2,P50221,58,98
3,P57073,242,318
4,P57073,370,409
...,...,...,...
2391,P17098,4,88
2392,Q9UJW7,12,113
2393,Q9UJW7,116,182
2394,Q9UJW7,184,247


,uniprotID,Start,End
0,Q96QS3,232,249
1,P57073,270,280
2,O14628,17,32
3,P50549,44,61
4,Q14209,399,436
...,...,...,...
716,Q8NAP8,67,88
717,P48742,279,289
718,Q9UI36,607,617
719,P17098,40,51


,uniprotID,Start,End
0,Q8WXT5,26,75
1,Q96MU6,18,83
2,O43313,630,690
3,O14978,79,120
4,O14978,191,234
...,...,...,...
541,P28698,173,226
542,Q53EQ6,500,550
543,Q07869,1,58
544,Q9H0D2,46,107


,uniprotID,Start,End
0,Q6P9G9,155,235
1,Q04206,418,502
2,Q9Y2G1,0,62
3,P43354,13,77
4,Q9ULD5,278,338
...,...,...,...
597,Q03701,943,1030
598,Q2M1K9,51,136
599,Q96LX8,0,107
600,Q9Y2D1,15,104


In [43]:
lambert_TFs = pd.read_csv("../data/LambertTFs.csv", index_col = 0)
lambert_TFs["uniprotID"] = lambert_TFs["GeneName"].str.split("|").str[1]
lambert_TFs["TF_length"] = lambert_TFs["ProteinSeq"].str.len()
lambert_TFs

,GeneName,ProteinSeq,uniprotID,TF_length
0,sp|P23511|NFYA_HUMAN,MEQYTANSNSSTEQIVVQAGQIQQQQQGGVTAVQLQTEAQVASASG...,P23511,347
1,sp|Q96QS3|ARX_HUMAN,MSNQYQEEGCSERPECKSKSPTLLSSYCIDSILGRRSPCKMRLLGA...,Q96QS3,562
2,sp|P31270|HXA11_HUMAN,MDFDERGPCSSNMYLPSCTYYVSGPDFSSLPSFLPQTPSSRPMTYS...,P31270,313
3,sp|P50221|MEOX1_HUMAN,MDPAASSCMRSLQPPAPVWGCLRNPHSEGNGASGLPHYPPTPFSFH...,P50221,254
4,sp|P57073|SOX8_HUMAN,MLDMSEARSQPPCSPSGTASSMSHVEDSDSDAPPSPAGSEGLGRAG...,P57073,446
...,...,...,...,...
1603,sp|P35227|PCGF2_HUMAN,MHRTTRIKITELNPHLMCALCGGYFIDATTIVECLHSFCKTCIVRY...,P35227,344
1604,sp|Q9BS34|ZN670_HUMAN,MDSVSFEDVAVAFTQEEWALLDPSQKNLYRDVMQEIFRNLASVGNK...,Q9BS34,389
1605,sp|P17098|ZNF8_HUMAN,MDPEDEGVAGVMSVGPPAARLQEPVTFRDVAVDFTQEEWGQLDPTQ...,P17098,575
1606,sp|Q9UJW7|ZN229_HUMAN,METLTSRHEKRALHSQASAISQDREEKIMSQEPLSFKDVAVVFTEE...,Q9UJW7,825


In [44]:
known_ADs= pd.read_csv("../../AD_predictor_project/Paper_AD_predictor/AD_predictor/output/known_ADs_considering_isoforms.csv")
known_ADs = known_ADs[known_ADs["TileType"] == "TF"]
known_ADs = known_ADs[known_ADs["uniprotID"].isin(lambert_TFs["uniprotID"])]
known_ADs

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
1,AEBP1,1088,1158,Q8IUX7,[['Q8IUX7']],NaN,Q8IUX7,Staller Activity Data,TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,NaN
2,AHCTF1,1445,1698,Q8WYP5,"[['Q8WYP5'], ['Q8WYP5']]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto / transcriptionalactivity...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,NaN
3,AHR,118,126,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,LLQALNGFV,NaN
4,AHR,266,268,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,FAI,NaN
5,AHR,532,848,P35869,"[['P35869'], ['P35869'], ['P35869']]",ENST00000242057 / nan / ENST00000242057,P35869 / P35869 / P35869,"DelRosso et al. / PMID: 8794892, Soto / DelRos...",TF,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
754,ZXDA,572,699,P98168,"[['P98168'], ['P98168']]",nan / nan,P98168 / P98168 / P98168,"PMID: 17493635, Soto / R4TA_regions.txt / acti...",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN
755,ZXDB,576,703,P98169,[['P98169']],NaN,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN
756,ZXDC,579,688,Q2QGD7,"[['Q2QGD7', 'Q2QGD7-2'], ['Q2QGD7', 'Q2QGD7-2']]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto / R4TA_regions.txt / acti...",TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,NaN
758,c-Jun/AP-1,238,257,P05412,[['P05412']],NaN,P05412,"Choi 2000 list, GSL",TF,ETPPLSPIDMESQERIKAER,NaN


In [68]:
def return_tp_fp(preds_input, knownADs):
    preds = preds_input.copy()
    AD_comparison_tools.add_col_contains_prediction(preds, knownADs, result_col_name = "KnownAD")
    TP_count = sum(preds["KnownAD"])
    FP_count = sum(preds["KnownAD"] == False)
    # print("True positives: " + str(TP_count) + "(" + str(np.round((100 * TP_count/ len(preds)), 1)) + "%)")
    # print("False positives: " + str(FP_count)+ "(" + str(np.round((100 * FP_count/ len(preds)), 1)) + "%)")
    return TP_count, FP_count

In [69]:
TP_count, FP_count = return_tp_fp(adhunter, known_ADs)

# Limit assessment to only predictions on TFs containing at least one known AD.
We limited our assessment of model performance by only looking at TFs that contained at least one entry on the GSL or Soto lists, assuming for a moment that all ADs on these TFs are known. Now the performance is X, which is much better. "

In [70]:
model_dict = {
    "TADA": tada,
    "ADHunter": adhunter,
    "ADPred": adpred,
    "Mechanistic": mechanistic,
    "PADDLE": paddle
} 

def eval_model(model_name):
    preds = model_dict[model_name]
    TF_lim_preds = preds[preds["uniprotID"].isin(known_ADs["uniprotID"])]
    num_preds = len(TF_lim_preds)
    benchmark_count = len(known_ADs)
    TP, FP =  return_tp_fp(preds_input = TF_lim_preds, knownADs = known_ADs)
    return model_name, num_preds, benchmark_count, TP, FP

In [71]:
model_results_df = pd.DataFrame(columns=["Predictor", "Predictions", "Benchmark", "True Positives", "False Positives"])
for model in model_dict.keys():
    model_results_df.loc[len(model_results_df)] = eval_model(model)
model_results_df

,Predictor,Predictions,Benchmark,True Positives,False Positives
0,TADA,840,599,446,394
1,ADHunter,655,599,370,285
2,ADPred,291,599,194,97
3,Mechanistic,218,599,148,70
4,PADDLE,271,599,219,52


In [78]:
model_results_df["False Negatives"] = model_results_df["Benchmark"] - model_results_df["True Positives"]
model_results_df["Precision"] = (model_results_df["True Positives"] / (model_results_df["True Positives"] + model_results_df["False Positives"])).round(2)
model_results_df["Recall"] = (model_results_df["True Positives"] / (model_results_df["True Positives"] + model_results_df["False Negatives"])).round(2)
model_results_df["F1 Score"] = (2 * (model_results_df["Precision"] * model_results_df["Recall"]) / (model_results_df["Precision"] + model_results_df["Recall"])).round(2)
model_results_df

,Predictor,Predictions,Benchmark,True Positives,False Positives,False Negatives,Precision,Recall,F1 Score
0,TADA,840,599,446,394,153,0.53,0.74,0.62
1,ADHunter,655,599,370,285,229,0.56,0.62,0.59
2,ADPred,291,599,194,97,405,0.67,0.32,0.43
3,Mechanistic,218,599,148,70,451,0.68,0.25,0.37
4,PADDLE,271,599,219,52,380,0.81,0.37,0.51
